# Envoyer un CSV dans Big Querry

**Auteur :**  Steve Caron  
**Date de création :** 2023/07/25  
**Présentation :** Ce notebook envoie le contenu d'un fichier csv dans BigQuerry


**Prérequis :** Un bucket gcp pour le stockage de données. (BUCKET_NAME)

**Inputs :** fichier d'entré: liste_station.csv 

**Params:**  
* INPUT_FILE_NAME: nom du fichier a traiter. DOIT être au format txt

In [2]:
from google.cloud import bigquery, storage
import os

In [3]:
TABLE_ID = "code-de-source.donnees_code_de_source.stations_meteo"

In [ ]:
BUCKET_NAME = "code_de_source_lake"
INPUT_FILE_NAME="liste_station.csv"



BUCKET_URI = f"gs://{BUCKET_NAME}"
INPUT_FILE = f"{BUCKET_URI}/{INPUT_FILE_NAME}"


# Telechargement du fichier depuis un bucket

In [5]:
# J'instancie un client storage
storage_client = storage.Client()
# Je précise le nom de bucket
bucket = storage_client.bucket(BUCKET_NAME)
#Dans un BLOB nommé mon_blob je charge le contenu de mon fichier
mon_blob = bucket.blob(INPUT_FILE_NAME)

# Je télecharge ensuite le contenu du BLOB nommé mon_blob dans un fichier
mon_blob.download_to_filename(INPUT_FILE_NAME)
print(f"Fichier CSV téléchargé depuis le bucket {BUCKET_NAME} vers {INPUT_FILE_NAME}.")

Fichier CSV téléchargé depuis le bucket bucket-tp-gcp vers liste_station.csv.


# Envoie vers BigTable

In [7]:
# J'instancie un Client BigQuerry
client = bigquery.Client()

# J'ouvre mon fichier qui est situé dans mon environnement de travail
with open(INPUT_FILE_NAME, "rb") as source_file:
    # Je creer une configuration de JOB en précisant le format et en sautant la premiere ligne  (header)
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        skip_leading_rows=1,
        source_format=bigquery.SourceFormat.CSV,
    )
    # Je lance mon job qui va envoyer le contenu de mon fichier source dans une table BigQuerry
    job = client.load_table_from_file(source_file, TABLE_ID, job_config=job_config)

job.result()

print(f"Chargement du fichier {INPUT_FILE_NAME} dans la table {TABLE_ID} terminé.")

Chargement du fichier liste_station.csv dans la table arcane-mission-393809.infoclimat.stations terminé.


# Suppression du fichier

In [6]:
if os.path.exists(INPUT_FILE_NAME):
    os.remove(INPUT_FILE_NAME)
    print(f"Le fichier{INPUT_FILE_NAME} a été correctement supprimé")
else:
    print(f"Impossible de supprimer le fichier {INPUT_FILE_NAME} car il n'existe pas")

Le fichierliste_station.csv a été correctement supprimé
